## Population estimation

1. Data acqisition
   1. Download world pop 100 m with year data same as event
   2. Download event data from ESA
   3. Download OSM boundaries for aois (admin level 4 (world 1, turkey 2, regions 3, subregions 4))
2. Preprocessing
   1. Extract aoi and builtUp files from zips according to the scripts below
   2. Cleanup builtUp: point on surface where the features are polygons
   3. Merge aoi files to one layer
   4. Merge builtUp files to one layer
   5. Dissolve aois
   6. Clip OSM boundaries to admin zones only containing the aois
3. *Calculation*: total population
   1. Clip pop raster with aoi_diss and warp to EPSG:3857
   2. Zonal Statistics: calculate sum for aois 
   3. Statistics by category: sum aoi sums by using column esmr_id -> `pop_total`
4. *Calculation*: population per admin
   1. Join OSM boundaries to aois
   2. Zonal statistics: calculate sum for aois
   3. Statistics by category: sum by using name of admin zones -> `pop_per_admin`

## Pre (1 day before)/Post (4 weeks after)

1. Preprocessing OSM buildings data
   1. Download OSM data according to the script below (pre/post)
   2. Calculate point on surface for osm polygon layer
   3. Merge new layer with osm point layer and reproject to EPSG:3857 -> `osm_centroids`
2. Pop per residential building (per cell for higher accuracy)
   1. Create grid in EPSG:3857 with 1 km² cell size (extent from aois)
   2. Cleanup pts (only residential)
   3. Count points in polygon (osm/pts in grid)
   4. Join pts_count and osm_count
   5. Delete cells where both counts are 0 (osm is 0 and pts is 0) -> `count per cell`
   6. Zonal statistics: calculate pop sum per cell (osm/pts count layers)
   7. Calculate pop sum / count -> `pop per building per cell`
3. Pop per residential building (per admin for comparison)
   1. Dissolve aois_admin
   2. Count points in polygon (osm/pts in aoi_admin_3857) -> `count_per_admin`
   3. Zonal statistics: calculate pop sum per admin
   4. Calculate pop sum / count -> `pop per building per admin`
4. *Analysis*: Completeness
   1. Calculate difference (osm count - pts count)
   2. Calculate deviation (osm count - pts count) / osm count * 100
   3. Assign classes: CASE WHEN "deviation" > 1 THEN 2 WHEN "deviation" < -1 THEN 0 ELSE 1 END
   4. Handle NULL values (if osm_count 0 and pts_count > 0 THEN class = 0)
   5. Generate class names: CASE WHEN "class" is 0 THEN 'less osm' WHEN "class" is 1 THEN 'similar' WHEN "class" is 2 THEN 'more osm' END -> `completeness`
   6. Statistics by categories: cell count for each class -> `cell_statistics`
   7. Visualize layers from before in map (left pre/right post)

HERE
1. *Discussion*: after the event much more OSM data (currentness)
   1. In osm_counts, assign timestamp (pre/post)
   2. Join both osm_counts (pre/post)
   3. Calculate difference (post - pre)
   4. Calculate deviation (post- pre) / post * 100 -> `currentness_osm`
   5. Visualize deviation: negative (one colour) and less to more change with continuous scale
2. *Discussion*: if aoi prone to disasters -> already more OSM data 
   1. Compare ohsome figures

*Attention: Aois in Indonesia not unique, so buffer before dissolving (0,0005 degrees) and adding unique id field after intersecting aois with osm boundaries necessary*

In [ ]:
#!/usr/bin/env python
# -*- coding: utf-8 -*-
"""Extracting files for Indonesia"""

import os
from glob import glob

print("Extracting data...")

in_dir = f"../data/ind/extract/"
for item in glob(in_dir + "*"):
    if item not in glob(in_dir + "*area_of_interest*") and item not in glob(in_dir + "*built_up*"):
        os.remove(item)
    if item in glob(in_dir + "*.kmz"):
        os.remove(item)

print("...finished.")

In [ ]:
#!/usr/bin/env python
# -*- coding: utf-8 -*-
"""Extracting files for Turkiye"""

import os
from glob import glob

print("Extracting data...")

in_dir = f"../data/tur/extract/"
for item in glob(in_dir + "*"):
    if item not in glob(in_dir + "*areaOfInterest*") and item not in glob(in_dir + "*builtUp*"):
        os.remove(item)
    if item in glob(in_dir + "*.kmz") or item in glob(in_dir + "*.xml"):
        os.remove(item)

print("...finished.")

In [ ]:
#!/usr/bin/env python
# -*- coding: utf-8 -*-
"""Download OSM data"""

import os
import sys
import pandas as pd
import geopandas as gpd
from pathlib import Path
from ohsome import OhsomeClient


client = OhsomeClient()
filter_buildings = "building in (yes, house, residential, appartements)"
country_list = ["ind", "tur"]

for country in country_list:

    in_dir = Path(f"../data/{country}")

    bpolys = gpd.read_file(in_dir / "mapdata.gpkg", layer = "aoi_diss")

    # specify timestamp as 1 day before event and 4 weeks afterwards
    if country == "ind":
        time_list = ["2018-09-27", "2018-10-25"]
    elif country == "tur":
        time_list = ["2023-02-05", "2023-03-05"]

    try:
        for time_c in time_list:
            response = client.elements.geometry.post(
                bpolys = bpolys, 
                filter = filter_buildings,
                time = time_c,
                properties = "metadata"
            )
            results_df = response.as_dataframe()
            results_df.to_file(in_dir / f"osm_{time_c}_buildings.geojson", driver = "GeoJSON")
    except Exception as err:
        print(f"Could not send request to ohsome API: {err}")
        sys.exit()
